# Make Detections with Model

- Now, let's see the performance of the model

In [3]:
import pickle
#import model
with open('GaitRecognition.pkl', 'rb') as f:
    model = pickle.load(f)

In [11]:
model

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=F

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd

In [13]:
#get the solutions of mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [14]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concate rows
            row = pose_row

            # Make Detections
            X = pd.DataFrame([row])
            predict_class = model.predict(X)[0]
            predict_prob = model.predict_proba(X)[0]#probability
            

            
            
            #Set the detection threshold
            if round(predict_prob[np.argmax(predict_prob)],2) > 0.7:

                
                # Grab ear coords, based on left ear
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640,480]).astype(int))

                cv2.rectangle(image, 
                              (coords[0], coords[1]+5), 
                              (coords[0]+len(predict_class)*20, coords[1]-30), 
                              (245, 117, 16), -1)
                cv2.putText(image, predict_class, coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            
                
                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, predict_class.split(' ')[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(predict_prob[np.argmax(predict_prob)],2))#heighest probability
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                print('===============================================')
                print('Detecting successful!')
                print('Detection: ', predict_class)
                print('===============================================')
            
            else:
                
                #print the results and the three probablities
                print('===============================================')
                print('Detecting failing!')
                print('Detection: Unknown')
                print('The most likely outcome is: ',predict_class)
                print('Here are the all probabilities:')
                print(predict_prob)
                print('===============================================')
                
                 # Grab ear coords, based on left ear
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640,480]).astype(int))

                cv2.rectangle(image, 
                              (coords[0], coords[1]+5), 
                              (coords[0]+len('Unknown')*20, coords[1]-30), 
                              (245, 117, 16), -1)
                cv2.putText(image, 'Unknown', coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
                
                
                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, 'Unknown'
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(predict_prob[np.argmax(predict_prob)],2))#heighest probability
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.37 0.11 0.   0.   0.02 0.   0.   0.37 0.   0.   0.   0.   0.   0.13
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.41 0.08 0.   0.   0.   0.   0.   0.4  0.   0.   0.   0.   0.   0.11
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.43 0.07 0.   0.   0.   0.   0.   0.41 0.   0.   0.   0.   0.   0.09
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.43 0.07 0.   0.   0.   0.   0.   0.41 0.   0.   0.   0.   0.   0.09
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  rj
Here are the all probabilities:
[0.41 0.06 0.   0.   0.01 0.   0.   0.44 0.   0.   0.   0.   0.   0.08
 0.

Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.58 0.12 0.   0.   0.01 0.   0.02 0.22 0.   0.   0.   0.   0.   0.05
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.59 0.12 0.   0.   0.01 0.   0.02 0.21 0.   0.   0.   0.   0.   0.05
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.62 0.11 0.   0.   0.01 0.   0.02 0.19 0.   0.   0.   0.   0.   0.05
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.62 0.11 0.   0.   0.01 0.   0.02 0.19 0.   0.   0.   0.   0.   0.05
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.62 0.11 0.   0.   0.01 0.   0.02 0.19 0.   0.   0.   0.   0.   0.05
 0

Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.62 0.11 0.   0.   0.01 0.   0.02 0.18 0.   0.   0.   0.   0.   0.06
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.62 0.1  0.   0.   0.01 0.   0.02 0.19 0.   0.   0.   0.   0.   0.06
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.62 0.1  0.   0.   0.01 0.   0.02 0.2  0.   0.   0.   0.   0.   0.05
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.62 0.1  0.   0.   0.01 0.   0.02 0.2  0.   0.   0.   0.   0.   0.05
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.63 0.1  0.   0.   0.01 0.   0.02 0.19 0.   0.   0.   0.   0.   0.05
 0

Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.58 0.12 0.   0.   0.01 0.   0.02 0.21 0.   0.   0.   0.   0.   0.06
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.58 0.12 0.   0.   0.01 0.   0.02 0.21 0.   0.   0.   0.   0.   0.06
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.58 0.12 0.   0.   0.01 0.   0.02 0.21 0.   0.   0.   0.   0.   0.06
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.58 0.12 0.   0.   0.01 0.   0.02 0.21 0.   0.   0.   0.   0.   0.06
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.58 0.12 0.   0.   0.01 0.   0.02 0.21 0.   0.   0.   0.   0.   0.06
 0

Detecting failing!
Detection: Unknown
The most likely outcome is:  zc
Here are the all probabilities:
[0.   0.   0.   0.   0.   0.01 0.   0.   0.04 0.09 0.   0.   0.   0.
 0.02 0.51 0.02 0.1  0.2  0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zc
Here are the all probabilities:
[0.   0.   0.   0.   0.   0.01 0.   0.   0.05 0.08 0.   0.   0.   0.
 0.02 0.53 0.02 0.1  0.18 0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zc
Here are the all probabilities:
[0.   0.   0.   0.   0.   0.01 0.   0.   0.05 0.08 0.   0.   0.   0.
 0.02 0.52 0.02 0.1  0.19 0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zc
Here are the all probabilities:
[0.   0.   0.   0.   0.01 0.01 0.   0.   0.05 0.08 0.   0.   0.   0.
 0.01 0.52 0.02 0.1  0.19 0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zc
Here are the all probabilities:
[0.   0.   0.   0.01 0.01 0.01 0.   0.   0.05 0.07 0.   0.   0.   0.
 0.01 0.53 0.01 0

...

Detecting failing!
Detection: Unknown
The most likely outcome is:  wl
Here are the all probabilities:
[0.   0.   0.   0.09 0.01 0.02 0.   0.   0.21 0.24 0.   0.   0.   0.
 0.02 0.13 0.05 0.17 0.04 0.02]
Detecting failing!
Detection: Unknown
The most likely outcome is:  syj
Here are the all probabilities:
[0.   0.   0.   0.09 0.01 0.03 0.   0.   0.22 0.22 0.   0.   0.   0.
 0.02 0.12 0.05 0.18 0.04 0.02]
Detecting failing!
Detection: Unknown
The most likely outcome is:  syj
Here are the all probabilities:
[0.   0.   0.   0.09 0.01 0.04 0.   0.   0.26 0.19 0.   0.   0.   0.
 0.02 0.12 0.05 0.17 0.03 0.02]
Detecting failing!
Detection: Unknown
The most likely outcome is:  syj
Here are the all probabilities:
[0.   0.   0.   0.08 0.   0.07 0.   0.   0.26 0.21 0.   0.01 0.   0.
 0.03 0.12 0.04 0.13 0.03 0.02]
Detecting failing!
Detection: Unknown
The most likely outcome is:  syj
Here are the all probabilities:
[0.   0.   0.   0.08 0.   0.07 0.   0.   0.26 0.22 0.   0.01 0.   0.
 0.02 0.11 0.

Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.01 0.03 0.   0.02 0.01 0.   0.19 0.17 0.   0.02 0.   0.
 0.05 0.06 0.05 0.27 0.05 0.07]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.01 0.03 0.   0.01 0.01 0.   0.18 0.17 0.   0.01 0.   0.
 0.05 0.07 0.05 0.29 0.05 0.07]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.01 0.03 0.   0.01 0.01 0.   0.19 0.15 0.   0.01 0.   0.
 0.06 0.06 0.05 0.31 0.05 0.06]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.01 0.04 0.   0.01 0.01 0.   0.18 0.13 0.   0.01 0.   0.
 0.05 0.08 0.06 0.33 0.04 0.05]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.01 0.04 0.01 0.01 0.01 0.   0.17 0.14 0.   0.   0.   0.
 0.05 0.08 0

Detecting failing!
Detection: Unknown
The most likely outcome is:  zc
Here are the all probabilities:
[0.   0.   0.   0.03 0.01 0.06 0.   0.   0.1  0.19 0.   0.   0.   0.
 0.1  0.21 0.02 0.12 0.14 0.02]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zl
Here are the all probabilities:
[0.   0.   0.   0.04 0.01 0.06 0.   0.   0.11 0.16 0.   0.   0.   0.
 0.12 0.14 0.03 0.14 0.18 0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.   0.05 0.01 0.08 0.   0.   0.08 0.19 0.   0.   0.   0.
 0.12 0.11 0.01 0.2  0.13 0.02]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.01 0.05 0.01 0.07 0.   0.   0.12 0.16 0.01 0.   0.   0.
 0.09 0.07 0.04 0.24 0.08 0.05]
Detecting failing!
Detection: Unknown
The most likely outcome is:  zjg
Here are the all probabilities:
[0.   0.   0.01 0.02 0.   0.06 0.   0.   0.12 0.15 0.   0.03 0.   0.
 0.08 0.1  0.0

Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.44 0.11 0.   0.   0.   0.   0.   0.4  0.   0.   0.   0.   0.   0.05
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.41 0.1  0.   0.   0.   0.   0.   0.4  0.   0.   0.   0.   0.   0.09
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  rj
Here are the all probabilities:
[0.39 0.1  0.   0.   0.   0.   0.   0.44 0.   0.   0.   0.   0.   0.07
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  rj
Here are the all probabilities:
[0.38 0.1  0.   0.   0.   0.   0.   0.45 0.   0.   0.   0.   0.   0.07
 0.   0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  rj
Here are the all probabilities:
[0.37 0.09 0.   0.   0.   0.   0.   0.47 0.   0.   0.   0.   0.   0.07
 0.  

Detecting failing!
Detection: Unknown
The most likely outcome is:  nhz
Here are the all probabilities:
[0.04 0.07 0.12 0.01 0.15 0.   0.19 0.06 0.   0.   0.06 0.03 0.11 0.13
 0.03 0.   0.   0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  nhz
Here are the all probabilities:
[0.04 0.07 0.11 0.02 0.07 0.   0.17 0.06 0.   0.   0.03 0.15 0.08 0.11
 0.04 0.   0.01 0.   0.   0.04]
Detecting failing!
Detection: Unknown
The most likely outcome is:  nhz
Here are the all probabilities:
[0.02 0.04 0.13 0.04 0.07 0.   0.2  0.04 0.   0.   0.05 0.16 0.05 0.05
 0.07 0.   0.03 0.   0.   0.05]
Detecting failing!
Detection: Unknown
The most likely outcome is:  wyc
Here are the all probabilities:
[0.02 0.03 0.12 0.06 0.06 0.   0.12 0.04 0.   0.   0.03 0.21 0.03 0.06
 0.07 0.   0.05 0.   0.   0.1 ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  wyc
Here are the all probabilities:
[0.02 0.03 0.11 0.06 0.03 0.   0.12 0.04 0.   0.   0.05 0.2  0.03 0.06
 0

# Add a new person

- After detecting, it cannot detect anyone. Of course! It can only detects  the 20 people from the CASIA Gait Recognition Dataset.
- In this case, we will get a group data from a new person first.

In [27]:
import csv

- We also have to set the dataset structure first.

In [30]:
num_coords = len(results.pose_landmarks.landmark)
landmarks = ['Class']
for val in range(1, num_coords+1):
    landmarks +=['x{}'.format(val), 'y{}'.format(val), 
                 'z{}'.format(val), 'v{}'.format(val)]
landmarks

['Class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [31]:
class_name = "HH"

- In order to prevent misoperation, we input the new person data in to the Newperson_full.csv

In [32]:
with open('Newperson_full.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [33]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concate rows
            row = pose_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('Newperson_full.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('New_person', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

- Before train the data, we creat a new file 'Data_with_newperson.csv' including the data from Data_coords_20.csv and Newperson_full.csv 
- Then, we should train the data just like we did before.

In [35]:
from sklearn.model_selection import train_test_split

In [37]:
df = pd.read_csv('Data_with_newperson.csv')
x = df.drop('Class', axis=1) # features
y = df['Class'] # target value
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [38]:
df

,Class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,fyc,0.879743,0.434765,-0.110076,0.998767,0.884691,0.423258,-0.112526,0.998340,0.887720,...,0.163869,0.858926,0.875535,0.894358,-0.074822,0.946482,0.815601,0.877874,0.101090,0.892224
1,fyc,0.875687,0.434991,-0.121108,0.998882,0.881445,0.424194,-0.120181,0.998495,0.884622,...,0.164102,0.858729,0.871622,0.894546,-0.070210,0.946372,0.789716,0.876701,0.109292,0.890800
2,fyc,0.865430,0.436499,-0.147009,0.998986,0.870713,0.426487,-0.145864,0.998635,0.873895,...,0.142747,0.857340,0.871901,0.894528,0.004848,0.945344,0.760859,0.870030,0.086790,0.892033
3,fyc,0.855601,0.439941,-0.162105,0.999083,0.861209,0.430437,-0.159268,0.998765,0.864392,...,0.127017,0.856817,0.872124,0.894467,0.018772,0.942292,0.739511,0.865657,0.067528,0.893602
4,fyc,0.839366,0.441057,-0.165840,0.999171,0.846083,0.431095,-0.166450,0.998884,0.849447,...,0.138057,0.851298,0.872283,0.894467,0.035043,0.942063,0.735843,0.867225,0.077553,0.894663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20163,HH,0.641455,-0.362151,-0.939049,0.779223,0.674466,-0.414684,-0.925853,0.596642,0.688190,...,0.720411,0.159885,0.481136,1.748212,0.417508,0.180281,0.333969,1.739886,0.341256,0.194935
20164,HH,0.631266,-0.309825,-0.909173,0.777431,0.664878,-0.366808,-0.908836,0.589613,0.679556,...,0.701877,0.151443,0.492800,1.756832,0.295402,0.166575,0.339450,1.739207,0.334632,0.183523
20165,HH,0.627786,-0.100703,-1.278459,0.793666,0.662882,-0.163038,-1.332970,0.618134,0.678376,...,1.153320,0.156843,0.499687,1.775002,0.840189,0.158356,0.343965,1.753538,0.844533,0.185823
20166,HH,0.652810,-0.419536,-0.984489,0.804504,0.687241,-0.478322,-0.992109,0.628129,0.702509,...,0.949112,0.146346,0.505092,1.806875,0.390820,0.146519,0.351697,1.768610,0.610711,0.172261


In [39]:
df.groupby('Class')['Class'].value_counts()

Class  Class
HH     HH       1079
fyc    fyc      1115
hy     hy       1000
ljg    ljg      1045
lqf    lqf      1050
lsl    lsl      1005
ml     ml        965
nhz    nhz      1026
rj     rj        883
syj    syj      1011
wl     wl        818
wq     wq        957
wyc    wyc       918
xch    xch       829
xxj    xxj      1016
yjf    yjf       934
zc     zc        839
zdx    zdx       891
zjg    zjg       985
zl     zl        857
zyf    zyf       945
Name: Class, dtype: int64

In [2]:
# Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Measuring Accuracy
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_roc_curve
import matplotlib.pyplot as plt 

# Data processing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import joblib

# Save and load models
import pickle

# Managing Datasets
import pandas as pd
# Extra Graphs
import seaborn as sns
# Maths
import numpy as np

#pipeline
from sklearn.pipeline import make_pipeline

In [41]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
    'knn':make_pipeline(StandardScaler(), KNeighborsClassifier()),
    'dt':make_pipeline(StandardScaler(), DecisionTreeClassifier()),
    'svc':make_pipeline(StandardScaler(),SVC())
}

In [42]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb', 'knn', 'dt', 'svc'])

In [43]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

C:\Users\huhon\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [44]:
model

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [45]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.8258139150553627
rc 0.6311353495290035
rf 0.9927284746322922
gb 0.957197157494629
knn 0.8998512642538423
dt 0.8972070732110395
svc 0.82796232027764


In [46]:
with open('Newperson_full.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

- Now, we can make detections with new model

# Make detections with new model

In [4]:
#import model
with open('Newperson_full.pkl', 'rb') as f:
    model = pickle.load(f)

In [5]:
model

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=F

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [7]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concate rows
            row = pose_row

            # Make Detections
            X = pd.DataFrame([row])
            predict_class = model.predict(X)[0]
            predict_prob = model.predict_proba(X)[0]#probability
            
            
            #Set the detection threshold
            if round(predict_prob[np.argmax(predict_prob)],2) > 0.7:

                
                # Grab ear coords, based on left ear
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640,480]).astype(int))

                cv2.rectangle(image, 
                              (coords[0], coords[1]+5), 
                              (coords[0]+len(predict_class)*20, coords[1]-30), 
                              (245, 117, 16), -1)
                cv2.putText(image, predict_class, coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            
                
                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, predict_class.split(' ')[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(predict_prob[np.argmax(predict_prob)],2))#heighest probability
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                print('===============================================')
                print('Detecting successful!')
                print('Detection: ', predict_class)
                print('===============================================')
            
            else:
                
                #print the results and the three probablities
                print('===============================================')
                print('Detecting failing!')
                print('Detection: Unknown')
                print('The most likely outcome is: ',predict_class)
                print('Here are the all probabilities:')
                print(predict_prob)
                print('===============================================')
                
                 # Grab ear coords, based on left ear
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640,480]).astype(int))

                cv2.rectangle(image, 
                              (coords[0], coords[1]+5), 
                              (coords[0]+len('Unknown')*20, coords[1]-30), 
                              (245, 117, 16), -1)
                cv2.putText(image, 'Unknown', coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
                
                
                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, 'Unknown'
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(predict_prob[np.argmax(predict_prob)],2))#heighest probability
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                print('===============================================')
            
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
D

Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.69 0.   0.   0.   0.   0.   0.08 0.   0.   0.21 0.01 0.   0.   0.
 0.   0.   0.   0.   0.01 0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.65 0.   0.   0.   0.   0.   0.09 0.   0.   0.22 0.01 0.   0.   0.
 0.   0.   0.   0.   0.03 0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.58 0.   0.   0.   0.   0.   0.09 0.   0.   0.28 0.01 0.   0.   0.
 0.   0.01 0.   0.01 0.02 0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.55 0.   0.   0.   0.   0.   0.09 0.   0.   0.3  0.01 0.   0.   0.
 0.   0.01 0.   0.02 0.02 0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.56 0.   0.   0.   0.   0.   0.11 0.   0.   0.24 0.01 0.   0.   

Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.   0.   0.   0.04 0.13 0.04 0.   0.12 0.   0.   0.   0.05 0.08 0.02
 0.01 0.37 0.   0.03 0.   0.   0.11]
Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.   0.   0.   0.05 0.13 0.04 0.   0.12 0.   0.   0.   0.07 0.06 0.04
 0.01 0.37 0.   0.01 0.   0.   0.1 ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.   0.   0.   0.04 0.12 0.04 0.   0.11 0.   0.   0.   0.07 0.06 0.04
 0.01 0.37 0.   0.02 0.   0.   0.12]
Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.   0.   0.   0.   0.11 0.04 0.   0.13 0.   0.   0.   0.09 0.07 0.04
 0.   0.38 0.   0.02 0.   0.   0.12]
Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.   0.   0.   0.02 0.1  0.03 0.   0.16 0.   0.   0.

Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.03 0.   0.01 0.   0.03 0.01 0.01 0.   0.   0.01 0.   0.01 0.02 0.01
 0.01 0.68 0.   0.08 0.01 0.   0.08]
Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.05 0.   0.01 0.   0.01 0.01 0.04 0.   0.   0.05 0.   0.01 0.02 0.01
 0.   0.58 0.   0.08 0.02 0.01 0.1 ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  yjf
Here are the all probabilities:
[0.08 0.   0.01 0.01 0.02 0.   0.05 0.   0.   0.16 0.   0.   0.   0.01
 0.   0.51 0.   0.02 0.01 0.02 0.1 ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  syj
Here are the all probabilities:
[0.2  0.   0.01 0.01 0.01 0.   0.13 0.   0.   0.37 0.   0.   0.   0.01
 0.   0.16 0.   0.02 0.02 0.03 0.03]
Detecting failing!
Detection: Unknown
The most likely outcome is:  syj
Here are the all probabilities:
[0.19 0.   0.01 0.01 0.01 0.   0.14 0.   0.   0.42 0.

Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.5  0.13 0.   0.05 0.   0.06 0.   0.06 0.05 0.02 0.   0.   0.01 0.01
 0.07 0.   0.   0.01 0.02 0.   0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.44 0.12 0.   0.09 0.   0.04 0.   0.13 0.02 0.01 0.   0.   0.02 0.
 0.07 0.02 0.   0.01 0.   0.   0.03]
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.64 0.13 0.01 0.05 0.01 0.05 0.   0.02 0.01 0.01 0.   0.   0.01 0.
 0.04 0.01 0.   0.01 0.   0.   0.  ]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.7  0.   0.   0.   0.01 0.   0.04 0.01 0.   0.03 0.   0.   0.   0.
 0.   0.04 0.   0.

...

Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
D

Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
D

Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
D

Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.24 0.34 0.05 0.   0.01 0.02 0.01 0.06 0.15 0.   0.   0.01 0.   0.
 0.09 0.   0.   0.01 0.   0.   0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  fyc
Here are the all probabilities:
[0.28 0.3  0.05 0.   0.01 0.03 0.01 0.04 0.13 0.   0.   0.01 0.   0.
 0.1  0.   0.   0.01 0.02 0.   0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.29 0.28 0.07 0.   0.01 0.03 0.01 0.04 0.12 0.   0.   0.01 0.   0.
 0.09 0.01 0.   0.01 0.02 0.   0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.35 0.27 0.06 0.   0.01 0.04 0.01 0.04 0.1  0.   0.   0.   0.   0.
 0.08 0.01 0.   0.01 0.01 0.   0.01]
Detecting failing!
Detection: Unknown
The most likely outcome is:  HH
Here are the all probabilities:
[0.35 0.23 0.05 0.   0.   0.03 0.01 0.04 0.13 0.   0.   0.   0. 

Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
Detecting successful!
Detection:  HH
D